In [ ]:
import findspark

In [ ]:
# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

In [ ]:
from pyspark.sql import SparkSession, functions as F

docker run -d --name some-mongo \
-p 27017:27017 \
-e MONGO_INITDB_ROOT_USERNAME=mongoadmin \
-e MONGO_INITDB_ROOT_PASSWORD=secret \
-v mongodb:/data/db \
mongo

    # Connect mongo shell and create user
    mongo -u mongoadmin -p secret

     use test
     db.createUser(
      {
        user: "train",
        pwd:  passwordPrompt(),   // or cleartext password
        roles: [ { role: "readWrite", db: "test" },
                 { role: "read", db: "reporting" } ]
      }
    )

In [ ]:
spark = SparkSession.builder \
.appName("MongoDB Example") \
.master("local[2]") \
.config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0") \
.config("spark.mongodb.auth.uri","mongodb://train:<password_here>@127.0.0.1:27017/test") \
.config("spark.mongodb.input.uri",
        "mongodb://127.0.0.1:27017/test.people") \
.config("spark.mongodb.output.uri","mongodb://127.0.0.1:27017/test.people") \
.getOrCreate()

# Read Data From File

In [ ]:
df = spark.read \
.option("inferSchema",True) \
.option("header", True) \
.csv("file:///home/train/datasets/simple_data.csv")

In [ ]:
df.show(3)

# Write to MongoDB

In [ ]:
df.write.format("mongo").mode("overwrite").option("database",
"test").option("collection", "people").save()

In [ ]:
# spark.stop()